In [1]:
# !ffmpeg -i input_video.mp4 frames/frame_%04d.png

In [2]:
# !ffmpeg -i output_video_fourth.mp4 frames_fourth/frame_%04d.png

In [3]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

In [4]:
def calculate_psnr_ssim_for_videos(reference_video, test_video, resize_to=None):
    cap_ref = cv2.VideoCapture(reference_video)
    cap_test = cv2.VideoCapture(test_video)

    psnr_values = []
    ssim_values = []
    
    while cap_ref.isOpened() and cap_test.isOpened():
        ret_ref, frame_ref = cap_ref.read()
        ret_test, frame_test = cap_test.read()

        if not ret_ref or not ret_test:
            break  

        if resize_to:
            frame_ref = cv2.resize(frame_ref, resize_to, interpolation=cv2.INTER_CUBIC)
            frame_test = cv2.resize(frame_test, resize_to, interpolation=cv2.INTER_CUBIC)

        frame_ref_gray = cv2.cvtColor(frame_ref, cv2.COLOR_BGR2GRAY)
        frame_test_gray = cv2.cvtColor(frame_test, cv2.COLOR_BGR2GRAY)

        # Compute PSNR
        mse = np.mean((frame_ref - frame_test) ** 2)
        if mse == 0:
            psnr = 100  
        else:
            psnr = 10 * np.log10(255**2 / mse)
        psnr_values.append(psnr)

        ssim_score = ssim(frame_ref_gray, frame_test_gray, data_range=255)
        ssim_values.append(ssim_score)

    cap_ref.release()
    cap_test.release()

    avg_psnr = np.mean(psnr_values) if psnr_values else None
    avg_ssim = np.mean(ssim_values) if ssim_values else None

    return avg_psnr, avg_ssim


# Quality of frames extracted by DLG

In [5]:
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')  # Perfect match
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim(img1, img2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    ssim_value = ssim(img1_gray, img2_gray, data_range=img1_gray.max() - img1_gray.min())
    return ssim_value

def calculate_metrics_for_folders(raw_folder, recon_folder):
    raw_images = sorted(os.listdir(raw_folder))  # Ensure order consistency
    recon_images = sorted(os.listdir(recon_folder))

    assert len(raw_images) == len(recon_images), 

    psnr_values = []
    ssim_values = []
    
    for raw_name, recon_name in zip(raw_images, recon_images):
        raw_path = os.path.join(raw_folder, raw_name)
        recon_path = os.path.join(recon_folder, recon_name)

        
        raw_img = cv2.imread(raw_path, cv2.IMREAD_COLOR)
        recon_img = cv2.imread(recon_path, cv2.IMREAD_COLOR)

        if raw_img is None or recon_img is None:
            print(f"Skipping {raw_name} or {recon_name}, as one of them is missing!")
            continue

        
        psnr_values.append(calculate_psnr(raw_img, recon_img))

        
        ssim_values.append(calculate_ssim(raw_img, recon_img))

    avg_psnr = sum(psnr_values) / len(psnr_values) if psnr_values else 0
    avg_ssim = sum(ssim_values) / len(ssim_values) if ssim_values else 0

    return avg_psnr, avg_ssim

In [6]:
raw_folder = "raw_frames_one_before_dlg"
recon_folder = "frames"
average_psnr, average_ssim = calculate_metrics_for_folders(raw_folder, recon_folder)

print(f"Average PSNR: {average_psnr:.2f} dB")
print(f"Average SSIM: {average_ssim:.4f}")

Average PSNR: 28.41 dB
Average SSIM: 0.7546


In [7]:
raw_folder = "raw_frames_two_before_dlg"
recon_folder = "frames_second"
average_psnr, average_ssim = calculate_metrics_for_folders(raw_folder, recon_folder)

print(f"Average PSNR: {average_psnr:.2f} dB")
print(f"Average SSIM: {average_ssim:.4f}")

Average PSNR: 29.49 dB
Average SSIM: 0.6478


In [8]:
raw_folder = "raw_frames_four_before_dlg"
recon_folder = "frames_fourth"
average_psnr, average_ssim = calculate_metrics_for_folders(raw_folder, recon_folder)

print(f"Average PSNR: {average_psnr:.2f} dB")
print(f"Average SSIM: {average_ssim:.4f}")

Average PSNR: 29.26 dB
Average SSIM: 0.4892


In [9]:
def quality_assesment(high_quality_video, low_quality_video_after_dlg, enhanced_video_with_1_ref, 
                      enhanced_video_with_2_ref, enhanced_video_with_no_ref):
    # calculation for method 1 --------------------------------------------------------
    # Downscale high-quality video to 128x128 for a fair reference
    psnr_ssim_high_quality_method1 = calculate_psnr_ssim_for_videos(high_quality_video, enhanced_video_with_1_ref, resize_to=(128, 128))
    # Upscale low-quality video to 128x128 as a simple baseline
    psnr_ssim_low_quality_method1 = calculate_psnr_ssim_for_videos(low_quality_video_after_dlg, enhanced_video_with_1_ref, resize_to=(128, 128))
    
    #calculation for method 2 --------------------------------------------------------
    # Downscale high-quality video to 128x128 for a fair reference
    psnr_ssim_high_quality_method2 = calculate_psnr_ssim_for_videos(high_quality_video, enhanced_video_with_2_ref, resize_to=(128, 128))
    # Upscale low-quality video to 128x128 as a simple baseline
    psnr_ssim_low_quality_method2 = calculate_psnr_ssim_for_videos(low_quality_video_after_dlg, enhanced_video_with_2_ref, resize_to=(128, 128))
    
    #calculation for method 3 --------------------------------------------------------
    # Downscale high-quality video to 128x128 for a fair reference
    psnr_ssim_high_quality_method3 = calculate_psnr_ssim_for_videos(high_quality_video, enhanced_video_with_no_ref, resize_to=(128, 128))
    # Upscale low-quality video to 128x128 as a simple baseline
    psnr_ssim_low_quality_method3 = calculate_psnr_ssim_for_videos(low_quality_video_after_dlg, enhanced_video_with_no_ref, resize_to=(128, 128))
    
    psnr_ssim_baseline = calculate_psnr_ssim_for_videos(high_quality_video, low_quality_video_after_dlg, resize_to=(128, 128))
    
    print("Results for using 1 reference")
    print(f"PSNR, SSIM (Enhanced vs. High-Quality Reference): {psnr_ssim_high_quality_method1[0]:.2f} dB, {psnr_ssim_high_quality_method1[1]}")
    print(f"PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): {psnr_ssim_low_quality_method1[0]:.2f} dB, {psnr_ssim_low_quality_method1[1]}")
    
    print('-----------------------------------------')
    
    print("Results for using multiple references")
    print(f"PSNR, SSIM (Enhanced vs. High-Quality Reference): {psnr_ssim_high_quality_method2[0]:.2f} dB, {psnr_ssim_high_quality_method2[1]}")
    print(f"PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): {psnr_ssim_low_quality_method2[0]:.2f} dB, {psnr_ssim_low_quality_method2[1]}")
    
    print('-----------------------------------------')
    
    print("Results for using NO references")
    print(f"PSNR, SSIM (Enhanced vs. High-Quality Reference): {psnr_ssim_high_quality_method3[0]:.2f} dB, {psnr_ssim_high_quality_method3[1]}")
    print(f"PSNR (Enhanced vs. Upscaled 32×32 Baseline): {psnr_ssim_low_quality_method3[0]:.2f} dB, {psnr_ssim_low_quality_method3[1]}")
    
    print('-----------------------------------------')
    
    print('reconstructed video quality only using DLG')
    print(f'PSNR, SSIM (High-Quality vs Upscaled 32×32 Baseline)  {psnr_ssim_baseline[0], psnr_ssim_baseline[1]}')


# Results for video 1

In [10]:
# Paths to videos
high_quality_video = "videos/video1.mp4"  # 240x320 original
low_quality_video_after_dlg = "videos_obtained_by_dlg/output_video.mp4"  # 32x32 original
enhanced_video_with_1_ref = "enhanced_videos/enhanced_video_one_with_1_ref.mp4"  # 128x128 enhanced
enhanced_video_with_2_ref = "enhanced_videos/enhanced_video_one_with_2_ref.mp4"
enhanced_video_with_no_ref = "enhanced_videos/enhanced_video_one_zero_ref.mp4"

quality_assesment(high_quality_video, low_quality_video_after_dlg, enhanced_video_with_1_ref, 
                      enhanced_video_with_2_ref, enhanced_video_with_no_ref)

Results for using 1 reference
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.68 dB, 0.22540953441628717
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 30.12 dB, 0.669345549154031
-----------------------------------------
Results for using multiple references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.68 dB, 0.2263069760913325
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 30.14 dB, 0.6743828010956185
-----------------------------------------
Results for using NO references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.75 dB, 0.21447792692339196
PSNR (Enhanced vs. Upscaled 32×32 Baseline): 29.52 dB, 0.6103029527575577
-----------------------------------------
reconstructed video quality only using DLG
PSNR, SSIM (High-Quality vs Upscaled 32×32 Baseline)  (28.400091463053652, 0.20256872377908608)


# Results for video 2

In [11]:
# Paths to videos
high_quality_video = "videos/video2.mp4"  # 240x320 original
low_quality_video_after_dlg = "videos_obtained_by_dlg/output_video_second.mp4"  # 32x32 original
enhanced_video_with_1_ref = "enhanced_videos/enhanced_video_two_with_1_ref.mp4"  # 128x128 enhanced
enhanced_video_with_5_ref = "enhanced_videos/enhanced_video_two_with_5_ref.mp4"
enhanced_video_with_no_ref = "enhanced_videos/enhanced_video_two_zero_ref.mp4"

quality_assesment(high_quality_video, low_quality_video_after_dlg, enhanced_video_with_1_ref, 
                      enhanced_video_with_5_ref, enhanced_video_with_no_ref)

Results for using 1 reference
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.76 dB, 0.22062955047439076
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 30.57 dB, 0.7006031482863804
-----------------------------------------
Results for using multiple references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.76 dB, 0.22045389195464757
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 30.57 dB, 0.6997962755360385
-----------------------------------------
Results for using NO references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.86 dB, 0.2311224597400999
PSNR (Enhanced vs. Upscaled 32×32 Baseline): 30.01 dB, 0.6841739678113785
-----------------------------------------
reconstructed video quality only using DLG
PSNR, SSIM (High-Quality vs Upscaled 32×32 Baseline)  (28.46833393409526, 0.19676092749753968)


# Results for video 4

In [12]:
# Paths to videos
high_quality_video = "videos/video5.mp4"  # 240x320 original
low_quality_video_after_dlg = "videos_obtained_by_dlg/output_video_fourth.mp4"  # 32x32 original
enhanced_video_with_1_ref = "enhanced_videos/enhanced_video_four_with_1_ref.mp4"  # 128x128 enhanced
enhanced_video_with_5_ref = "enhanced_videos/enhanced_video_four_with_5_ref.mp4"
enhanced_video_with_no_ref = "enhanced_videos/enhanced_video_four_zero_ref.mp4"

quality_assesment(high_quality_video, low_quality_video_after_dlg, enhanced_video_with_1_ref, 
                      enhanced_video_with_5_ref, enhanced_video_with_no_ref)

Results for using 1 reference
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.95 dB, 0.3176517925743453
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 32.76 dB, 0.8770833305914224
-----------------------------------------
Results for using multiple references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.95 dB, 0.31782212612839056
PSNR, SSIM (Enhanced vs. Upscaled 32×32 Baseline): 32.77 dB, 0.8780426687434114
-----------------------------------------
Results for using NO references
PSNR, SSIM (Enhanced vs. High-Quality Reference): 28.74 dB, 0.29563306725180477
PSNR (Enhanced vs. Upscaled 32×32 Baseline): 31.37 dB, 0.7977580691186403
-----------------------------------------
reconstructed video quality only using DLG
PSNR, SSIM (High-Quality vs Upscaled 32×32 Baseline)  (28.632599289309415, 0.2880909293180442)


# Bit depth check

In [13]:
def check_png_bit_depth(filename):
    with open(filename, "rb") as f:
        f.seek(24)  # Jumping to byte 24 (zero-based index)
        bit_depth = int.from_bytes(f.read(1), "big")  # Reading 1 byte for bit depth
        color_type = int.from_bytes(f.read(1), "big")  # Reading 1 byte for color type

    color_types = {
        0: "Grayscale",
        2: "RGB",
        3: "Indexed (Colormapped)",
        4: "Grayscale + Alpha",
        6: "RGBA"
    }

    color_desc = color_types.get(color_type, "Unknown")

    print(f"Bit Depth: {bit_depth}")
    print(f"Color Type: {color_type} ({color_desc})")

    if bit_depth == 8:
        print(" This is an 8-bit PNG.")
    else:
        print(" This is NOT an 8-bit PNG.")

# Example usage
check_png_bit_depth("frames/frame_0001.png")


Bit Depth: 8
Color Type: 2 (RGB)
 This is an 8-bit PNG.
